In this cell takes place the import of the necessary tools.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
from datetime import datetime
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

Here is reading the dataset

In [ ]:
url = "/datacov.csv"
data = pd.read_csv(url)
data.info()
print(f"Read in {len(data)} rows")

In [ ]:
vacines1 = x['FAB_COV_1'].unique()
vacines2 = x['FAB_COV_2'].unique()
vacines_lote = x['LAB_PR_COV'].unique()
print(vacines1)
print("\n\n\n")
print(vacines2)
print("\n\n\n")
print(vacines_lote)

Here manually droping the columns containing irrelevant and redundant data 

In [ ]:
data.drop(['DT_NASC', 'TP_IDADE','CS_ESCOL_N', 'CO_PAIS', 'DT_DIGITA', 'FNT_IN_COV', 'ID_PAIS','LOTE_2_COV','FAB_COV_1', 
        'AVE_SUINO', 'POS_PCRFLU', 'POS_PCROUT', 'HISTO_VGM', 'PCR_SARS2', 'ID_REGIONA','ID_MUNICIP','FAB_COV_2','LAB_PR_COV',
        'ID_UNIDADE', 'ID_RG_RESI', 'ID_MN_RESI', 'ID_RG_INTE', 'ID_MN_INTE', 'COD_IDADE', 'LOTE_1_COV',
          ], axis=1, inplace=True)
data.info()

Droping the lines with contain data of type 3 and 9

In [ ]:
aux = data[(data['EVOLUCAO'] == 3) | (data['EVOLUCAO'] == 9)].index
data.drop(aux, inplace=True)
data.info()

In [ ]:
x = data
x.head()

keeping only state of Parana lines

In [ ]:
aux = x[(x['SG_UF_NOT'] != 'PR')].index
x.drop(aux , inplace=True)
x.drop(['SG_UF_NOT','SG_UF','SG_UF_INTE'], axis=1, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

keeping only diagnosis of COVID19 lines

In [ ]:
#mantém somente diagnóstico de COVID
aux = x[(x['CLASSI_FIN'] != 5)].index
x.drop(aux , inplace=True)
x.drop(['CLASSI_FIN'], axis=1, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

In [ ]:
number_columns = x[['SEM_NOT','SEM_PRI','NU_IDADE_N','CS_GESTANT','OBES_IMC']]
x.drop(['SEM_NOT','SEM_PRI','NU_IDADE_N','CS_GESTANT','OBES_IMC'], axis=1, inplace=True)
x.replace(9, np.nan)
x = pd.concat([x, number_columns], axis=1)

In [ ]:
x.head()

In [ ]:
#remoção das colunas com 50% de dados faltantes
x.dropna(axis=1, how='any', thresh=9111, subset=None, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

In [ ]:
#Remoção das linhas com 50% de dados faltantes
x.dropna(axis=0, thresh=38, subset=None, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

In [ ]:
x.head()

In [ ]:
x['DT_NOTIFIC'] = pd.to_datetime(x['DT_NOTIFIC'])
x['DT_ENCERRA'] = pd.to_datetime(x['DT_ENCERRA'])
x['TEMP_T'] = (x['DT_ENCERRA'] - x['DT_NOTIFIC']).dt.days

In [ ]:
x.head()

In [ ]:
x['DT_SIN_PRI1'] = pd.to_datetime(x['DT_SIN_PRI'],infer_datetime_format=True, errors='coerce' ).apply(datetime.toordinal)
x['DT_INTERNA1'] = pd.to_datetime(x['DT_INTERNA'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DT_COLETA1'] = pd.to_datetime(x['DT_COLETA'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DT_PCR1'] = pd.to_datetime(x['DT_PCR'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DT_EVOLUCA1'] = pd.to_datetime(x['DT_EVOLUCA'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DOSE_1_COV1'] = pd.to_datetime(x['DOSE_1_COV'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x['DOSE_2_COV1'] = pd.to_datetime(x['DOSE_2_COV'], infer_datetime_format=True, errors='coerce').apply(datetime.toordinal)
x["CS_SEXO"] = x["CS_SEXO"].replace({"F": 0, "M": 1})
x = x.drop(['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_INTERNA', 'DT_COLETA', 'DT_PCR', 'DT_EVOLUCA', 'DT_ENCERRA', 'DOSE_1_COV', 'DOSE_2_COV'], axis=1)

In [ ]:
x.head()

In [ ]:
dadosnumericos = x[['SEM_NOT', 'SEM_PRI', 'NU_IDADE_N', 'CS_GESTANT', 'DT_SIN_PRI1', 'DT_INTERNA1', 'DT_COLETA1',
                   'DT_PCR1', 'DT_EVOLUCA1', 'TEMP_T', 'DOSE_1_COV1', 'DOSE_2_COV1']]

x.drop(columns=['SEM_NOT', 'SEM_PRI', 'NU_IDADE_N', 'CS_GESTANT', 'DT_SIN_PRI1', 'DT_INTERNA1', 'DT_COLETA1', 'DT_PCR1', 'DT_EVOLUCA1', 'DOSE_1_COV1', 'DOSE_2_COV1'], inplace=True)

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
nome = dadosnumericos.columns
imputer = imputer.fit(dadosnumericos)
dadosnumericos = imputer.transform(dadosnumericos)
scaler = MinMaxScaler()
dadosnumericos = scaler.fit_transform(dadosnumericos)
dadosnumericos = pd.DataFrame(dadosnumericos)
dadosnumericos.columns = nome


imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
nomes = x.columns
imputer = imputer.fit(x)
x = imputer.transform(x)
x = pd.DataFrame(x)
x.columns = nomes
x = pd.concat([x, dadosnumericos], axis=1)
x.loc[x['VACINA_COV'] == 2.0, ['DOSE_1_COV1', 'DOSE_2_COV1']] = 0

In [ ]:
y = x['EVOLUCAO']
x = x.drop(columns= 'EVOLUCAO')
pd.set_option('display.max_rows', None)
x.iloc[0:20, :]

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=5, random_state=43)

score = cross_val_score(clf, x, y, cv=10)
print(f"Accuracy of floresta aleátoria Classifier is:{score}")
print('Média: {:.2} | Desvio: {:.2}'.format(np.mean(score), np.std(score)))